# Using dash
The scope of this notebook is to learn how to use dash

### Import of packages

In [41]:
#General
import pandas as pd
import numpy as np
#dash related
import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.graph_objs as go


### Import a datset to use for visualizations
We will be using the fifa 2017 dataset posted on Kaggle (https://www.kaggle.com/artimous/complete-fifa-2017-player-dataset-global)

In [42]:
df = pd.read_csv('Fifa_forDash.csv')
df.sort_values('Rating', inplace=True, ascending=False)
df = df.reset_index().drop('index', axis = 1)

In [43]:
df.head()

,Name,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,...,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes
0,Cristiano Ronaldo,Portugal,LS,7.0,Real Madrid,LW,7.0,07/01/2009,2021.0,94,...,90,81,76,85,88,14,7,15,11,11
1,Lionel Messi,Argentina,RW,10.0,FC Barcelona,RW,10.0,07/01/2004,2018.0,93,...,88,89,90,74,85,14,6,15,11,8
2,Neymar,Brazil,LW,10.0,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,...,77,79,84,81,83,15,9,15,9,11
3,Luis Suárez,Uruguay,LS,9.0,FC Barcelona,ST,9.0,07/11/2014,2021.0,92,...,86,86,84,85,88,33,27,31,25,37
4,Manuel Neuer,Germany,GK,1.0,FC Bayern,GK,1.0,07/01/2011,2021.0,92,...,16,14,11,47,11,91,89,95,90,89


In [44]:
Nationalities = df.groupby('Nationality')['Name'].nunique().sort_values(ascending = False).index
Nat_counts = df.groupby('Nationality')['Name'].nunique().sort_values(ascending = False).values

In [52]:
colorfunct = {True:'#7b3294',False:'#7fbf7b'}


## Dash with an interactive dropdown that defines the content of the second plot

In [54]:
#create a dash object
app = dash.Dash()

#data for first plot
data1 = [{'x': list(range(0,len(Nationalities))), 
         'y': Nat_counts, 
         'type': 'bar',
         'text': Nationalities,
         'name':'rating' }]

scatterrandom = [go.Scatter(
                x = pd.to_numeric(df.Height.str.replace('cm', '')),
                y = pd.to_numeric(df.Weight.str.replace('kg', '')),
                text = df.Name,
                mode='markers',
                marker=dict(
                            size='5',
                            color = (df.Nationality==selected).map(colorfunct)
                            )
)]

#data for second plot
df.sort_values('Rating', ascending=False)
selected = 'England'


#layout of the dash
app.layout = html.Div([
    #first plot (half width)
    html.Div([dcc.Graph(
              id='example-graph',
              figure={
                'data': data1,
                'layout': {'title': 'Number of players per nationality'}
               }
              )],
              style={'width': '48%', 'float': 'left', 'display': 'inline-block'}
            ),
    #second plot (half width)
    html.Div([dcc.Graph(
              id='scatterrandom',
              figure={
                  'data':scatterrandom,
                  'layout' : go.Layout(title='Weight VS Height',
                                        xaxis={'title': 'Height'},
                                        yaxis={'title': 'Weight'},
                                        hovermode='closest'
                                        )
                    }
             )], 
             style={'width': '48%', 'float': 'left', 'display': 'inline-block'}
            ),
    
    #dropdown with nationalities
    html.Div([
                dcc.Dropdown(
                    id='dropdown-nationality',
                    options=[{'label': i, 'value': i} for i in Nationalities],
                    value='England'
                ),
            #third plot 
                dcc.Graph(id='example-graph2')
                ##this plot is built in realtime using the output from a function
             ],
        style={'width': '48%', 'float': 'left', 'display': 'inline-block'} #specifies that is half width
    ),                                                                  #and should appear on the left

    #fourth plot 
    html.Div([dcc.Graph(id='example-graph3')
        
    ],
        style={'width': '48%', 'float': 'right', 'display': 'inline-block'}
    )
]
)
                     
###What is updated when the dash is run? FOR PLOT 2
@app.callback(
    dash.dependencies.Output('example-graph2', 'figure'),     ### This output is updated in realtime using 
                                                              ### the function 'update_graph'
     [dash.dependencies.Input('dropdown-nationality', 'value')])   ###The input from 'dropdown-nationality' is read

#Function to update the plot as called by dash.dependencies.Output
def update_graph(selected):
    df_subset = df[df['Nationality'] == selected]
    df_subset.reset_index(inplace=True)
    
    #Defines the data for the updated plot
    data2 = [{'x': df_subset.index, 
             'y': df_subset.Rating, 
             'type': 'scatter',
             'text': df_subset.Name,
             'name':'rating' }
            ]
    #Defines the updated layout of the plot
    layout = {'title': 'Player ratings for ' + selected,
                   'yaxis': dict(
                                autotick=False,
                                ticks='outside',
                                range=[40, 95],
                                dtick=5,
                                ticklen=8,
                                tickwidth=4,
                                tickcolor='#000'
                                )
                  }
    #return a dictionary that defines the plot to be updated
    return {
        'data': data2,
        'layout': layout
    }


###What is updated when the dash is run? FOR PLOT 3
@app.callback(
    dash.dependencies.Output('example-graph3', 'figure'),     ### This output is updated in realtime using 
                                                              ### the function 'update_graph'
     [dash.dependencies.Input('dropdown-nationality', 'value')])   ###The input from 'dropdown-nationality' is read

#Function to update the plot as called by dash.dependencies.Output
def update_graph(selected):
    df_subset = df[df['Nationality'] == selected]
    df_subset.groupby('Rating')['Name'].count()    
    grouped = df_subset.groupby('Rating')['Name'].count()

    #Defines the data for the updated plot
    data3 = [go.Bar(
            x = grouped.index,
            y = grouped.values,
            marker = dict(color = '#efedf5',
                          line=dict(color='#756bb1',
                                    width=1
                                   )
                         )
            )
            ]
            
            
    #Defines the updated layout of the plot
    layout = go.Layout(
            title='Player ratings distribution for ' + selected,
            xaxis={'title': 'Rating'},
            yaxis={'title': 'Count'},
            hovermode='closest',
        )
    
    #return a dictionary that defines the plot to be updated
    return {
        'data': data3,
        'layout': layout
    }



#To run the dash!    
if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2017 22:38:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2017 22:38:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2017 22:38:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2017 22:38:26] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2017 22:38:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2017 22:38:26] "POST /_dash-update-component HTTP/1.1" 200 -


In [12]:
for col in df.columns:
    print(col)

Name
Nationality
National_Position
National_Kit
Club
Club_Position
Club_Kit
Club_Joining
Contract_Expiry
Rating
Height
Weight
Preffered_Foot
Birth_Date
Age
Preffered_Position
Work_Rate
Weak_foot
Skill_Moves
Ball_Control
Dribbling
Marking
Sliding_Tackle
Standing_Tackle
Aggression
Reactions
Attacking_Position
Interceptions
Vision
Composure
Crossing
Short_Pass
Long_Pass
Acceleration
Speed
Stamina
Strength
Balance
Agility
Jumping
Heading
Shot_Power
Finishing
Long_Shots
Curve
Freekick_Accuracy
Penalties
Volleys
GK_Positioning
GK_Diving
GK_Kicking
GK_Handling
GK_Reflexes


In [36]:
(df['Nationality']==selected).map(colorfunct)

0        blue
1        blue
2        blue
3        blue
4        blue
5        blue
6        blue
7        blue
8        blue
9        blue
10       blue
11       blue
12       blue
13       blue
14       blue
15       blue
16       blue
17       blue
18       blue
19       blue
20       blue
21       blue
22       blue
23       blue
24       blue
25       blue
26       blue
27       blue
28       blue
29       blue
         ... 
17558     red
17559    blue
17560     red
17561     red
17562    blue
17563    blue
17564    blue
17565    blue
17566    blue
17567    blue
17568     red
17569     red
17570     red
17571    blue
17572    blue
17573    blue
17574    blue
17575     red
17576    blue
17577    blue
17578     red
17579     red
17580     red
17581     red
17582    blue
17583    blue
17584    blue
17585     red
17586    blue
17587     red
Name: Nationality, Length: 17588, dtype: object